In [1]:
import os, sys, json
from sys import platform
_base_path = '\\'.join(os.getcwd().split('\\')[:-1]) + '\\' if platform == 'win32' else '/'.join(os.getcwd().split('/')[:-1]) + '/'
sys.path.append(_base_path)
from src.helpers.utils import get_config
from src.modules.explainability import get_explanations

In [2]:
_config = get_config()
json_path = os.path.join(_base_path, _config.get('JSON_FOLDER'))
prompt_path = os.path.join(_base_path, _config.get('PROMPT_FOLDER'))
if platform == 'win32':
	json_path = json_path.replace('/', '\\')
	prompt_path = prompt_path.replace('/', '\\')

In [3]:
PROMPT = "Using the provided JSON data from a brain MRI study, please generate a comprehensive report detailing the segmentation results of a brain tumor. Describe the tumor's presence in various brain regions according to the Julich-Brain Atlas, noting the percentages of the tumor within each region and the impact on the region. While discussing the confidence of the segmentation and the model used, keep the explanations high-level and avoid overly technical details. The goal is to enhance the explainability of the AI segmentation to support clinical decision-making in a manner that can be easily understood by medical professionals. "

PROMPT = "Using the provided JSON data from a brain MRI study, please generate a comprehensive report detailing the segmentation results of a brain tumor. The report should be structured as follows: 1. **Summary of Tumor Segmentation**: Provide an overview of the tumor's distribution across various regions of the brain according to the Julich-Brain Atlas. Discuss the semantic segmentation of the tumor into different categories (tumor core, peritumoral edema, GD-enhancing tumor) and their implications. 2. **Detailed Regional Impact**: For each region where the tumor is present, describe the percentage of the tumor within the region and the percentage of the region affected by the tumor. Discuss the known roles and functions of these regions in the brain and how the presence of the tumor might impact these functions. 3. **Semantic Segmentation**: Discuss the different aspects of the tumor identified by the semantic segmentation (tumor core, peritumoral edema, GD-enhancing tumor) and their potential clinical implications. 4. **Clinical Implications**: Discuss how the insights gained from the segmentation results could guide clinical decision-making, including potential treatment strategies and areas for further investigation. While discussing the confidence of the segmentation and the model used, keep the explanations high-level and avoid overly technical details. The goal is to enhance the explainability of the AI segmentation to support clinical decision-making in a manner that can be easily understood by medical professionals. "

In [4]:
with open(os.path.join(json_path, 'SegResNet_sample_2.json'), 'r') as f:
	d = json.load(f)
	PROMPT += str(d)

In [5]:
output1 = get_explanations(
	model_id = 'BioMistral/BioMistral-7B',
	prompt = PROMPT,
	write_to_file = True,
	output_path = prompt_path,
	output_length = 1024,
	verbose = True
)

The brain MRI scan from the provided JSON data revealed a large, heterogeneous mass in the left temporal lobe. 
The tumor was found to be predominantly located in the temporal-to-parietal gap (gap map), with smaller portions extending into the area hOc4la (lateral occipital cortex), area hOc4lp (lateral occipital cortex), area hIP4 (inferior parietal cortex), and area PGp (posterior superior gyrus). 
The tumor affected 49.22% of the temporal-to-parietal gap, 14.64% of the area hOc4la, 12.62% of the area hOc4lp, 6.36% of the area hIP4, and 4.98% of the area PGp. 
The tumor was segmented into three distinct regions: tumor core, peritumoral edema, and GD-enhancing tumor. 
The tumor core was the largest region, comprising 40.12% of the entire tumor. 
The peritumoral edema region was the second largest, accounting for 38.74% of the tumor, while the GD-enhancing tumor region was the smallest, constituting 21.14% of the tumor.


In [6]:
output2 = get_explanations(
	model_id = 'meta-llama/Meta-Llama-3-8B-Instruct',
	prompt = PROMPT,
	write_to_file = True,
	output_path = prompt_path,
	output_length = 1024,
	verbose = True
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

**Summary of Tumor Segmentation**

The tumor segmentation analysis reveals a significant presence of the tumor in various regions of the brain, primarily affecting the right hemisphere. 
The tumor is predominantly distributed in the Temporal-to-Parietal region (49.22%), followed by the Area hOc4la (LOC) region (14.64%), and then the Area hOc4lp (LOC) region (12.62%). 
The tumor also affects the Area hIP4 (IPS) region (6.36%) and the Area PGp (IPL) region (4.98%) to a lesser extent.

The semantic segmentation of the tumor into different categories (tumor core, peritumoral edema, GD-enhancing tumor) provides valuable insights into the tumor's characteristics. 
The tumor core is predominantly red, indicating a high degree of cellularity and malignant potential. 
The peritumoral edema is yellow, suggesting inflammation and tissue damage. 
The GD-enhancing tumor is green, indicating a high level of blood-brain barrier disruption and potential for treatment response.

**Detailed Regional Imp